# Linear Chain Gaussian Bayesian Nets

In [1]:
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt
from collections import defaultdict
import pandas as pd
import os as os
%matplotlib inline
from sklearn.linear_model import LinearRegression

In [2]:
def get_data_matrix(file_path):    
    '''Strip out the first row and first column and return data matrix'''
    data = np.loadtxt(file_path, delimiter=',', skiprows=1)
    return data[:,1:]

In [3]:
def create_matrix_for_regression(sensor_readings):
    '''
    I/P : a np array [1,2,3,4]
    O/P : a np matrix of form 
        |1 2|
        |2 3|
        |3 4|
    '''
    matrix = []
    for i in range(len(sensor_readings) - 1):
        matrix.append([sensor_readings[i],sensor_readings[i+1]])
       
    return np.array(matrix)

In [4]:
create_matrix_for_regression(np.array((1,2,3,4)))

array([[1, 2],
       [2, 3],
       [3, 4]])

In [5]:
def compute_regression_coeffs(data_matrix):
    '''
    I/P : a data_matrix
    O/P : The coefficients of the regressed model with a polynomial order of 1.
    '''    
    row_len = len(data_matrix[0])
    
    
    X_train = data_matrix[:,0:row_len-1]        
    y_train = data_matrix[:,row_len-1:]
    
    regr = LinearRegression()
    
    regr.fit(X_train, y_train)
    
    coefs = [] 
    
    #beta_0
    coefs.append(regr.intercept_[0])
    
    #Remaining betas
    for x in regr.coef_[0]:
        coefs.append(x)
    
    return coefs

In [6]:
print '2 + 3x matrix\n'
dm = np.array([[2,8],[5,17],[15,47],[45,137]])
print dm

print '\nBetas \n'
print compute_regression_coeffs(dm)

2 + 3x matrix

[[  2   8]
 [  5  17]
 [ 15  47]
 [ 45 137]]

Betas 

[2.0000000000000142, 2.9999999999999991]


In [7]:
def get_cond_variance(actual_lst,predicted_lst):
    '''
    I/P : Actual and predicted readings.
    O/P : Conditional Variance, which is the variance of errors.    
    '''
    
    actual_lst = np.array(actual_lst)
    predicted_lst = np.array(predicted_lst)
    
    error = actual_lst - predicted_lst
    
    return np.var(error)

In [8]:
get_cond_variance([1,2],[3,9])

6.25

In [9]:
def get_mean(beta_0,beta_1,prev_mean):
    '''Return current mean based on the the previous node''s mean'''
    '''Current mean , Mu_i = B_0 + B_1*Mu_i-1'''
    
    return beta_0 + (beta_1 * prev_mean)

In [10]:
def get_var(cond_var,beta_1,prev_sigma):
    '''Return current var based on the the previous node''s var'''
    '''Current var , sigma_i = cond_var + (B_1^2)*sigma_i-1'''
    
    return cond_var + ((beta_1**2) * prev_sigma)

##Modelling

###Model I : Totally 5 parameters per sensor.<br>
$1.\mu_{i}$ : Mean of all readings of a sensor<br>
$2.\sigma_{i}$ : Variance of all readings of a sensor<br>
$3.\beta_{0}$ : Regression coeff1<br>
$4.\beta_{1}$ : Regression coeff2<br>
$5.\sigma^{2}$ : Conditional Variance<br>


In [11]:
def get_model_1_params(data_set):
    '''Compute the Linear chain Gaussian Model parameters at each sensor and return the modelled matrix
       Ever sensor will have its own (mu_i,var_i,b_0,b_1,sigma**2) tuples
    '''
    no_of_sensors = data_set.shape[0]
    no_of_time_stamps = data_set.shape[1]
    
    modelled_mat = []
    
    for s_id in range(0,no_of_sensors):
        #Compute 5 params [START]        
        
        mu_i = np.mean(data_set[s_id])
        var_i = np.var(data_set[s_id])
        
        reg_mat = create_matrix_for_regression(data_set[s_id])        
        betas = compute_regression_coeffs(reg_mat)
                        
        true_y = reg_mat[:,1:2]
        x = reg_mat[:,0:1]
        
        pred_y = betas[0] + betas[1]*x
        
        cond_var = get_cond_variance(true_y,pred_y)
        #Compute 5 params [END]
        
        modelled_mat.append((mu_i,var_i,betas[0],betas[1],cond_var))
        
    return modelled_mat

In [95]:
dm = get_data_matrix('intelTemperatureTrain.csv')
model_1_params = get_model_1_params(dm)
print len(model_1_params)
print model_1_params[0]

50
(21.613888888888887, 6.8405015432098768, 0.80555773651311213, 0.96339227181482645, 0.42905098291130261)


###Model II  : <br>
$1.\mu_{i}$ : Mean of time 0 reading sensor across 3 days<br>
$2.\sigma_{i}$ : Variance of time 0 reading sensor across 3 days<br>
$3.\beta^{i}_{0}$ : Regression coeff1 for $i^{th}$ node<br>
$4.\beta^{i}_{1}$ : Regression coeff2 for $i_{th}$ node<br>
$5.\sigma^{2}_{i}$ : Conditional Variance<br>

In [14]:
def get_model_2_matrix(data_set,train_days):
    '''Fit the model 2 for the train set'''
    no_of_sensors = data_set.shape[0]
    no_of_time_stamps = data_set.shape[1]/train_days
    
    modelled_matrix = np.zeros((no_of_sensors,no_of_time_stamps),dtype=tuple)
            
    for s_id in range(0,no_of_sensors):    
        
        consolidated_first_reading = [data_set[s_id][no_of_time_stamps*0],data_set[s_id][no_of_time_stamps*1],data_set[s_id][no_of_time_stamps*2]]
        mu_i =  np.mean(consolidated_first_reading)
        sigma_i = np.var(consolidated_first_reading)
        
        modelled_matrix[s_id,0] = (mu_i,sigma_i)
        
        #compute betas from timestamp 1 onwards. ignore 0.5
        #print 'sensor ', s_id , ' -->'
        for time_stamp in range(0,no_of_time_stamps-1):
            
            day1_idx = time_stamp + (no_of_time_stamps*0)
            day2_idx = time_stamp + (no_of_time_stamps*1)
            day3_idx = time_stamp + (no_of_time_stamps*2)
            
            x_day_1 = data_set[s_id,day1_idx]
            y_day_1 = data_set[s_id,day1_idx+1]
            
            x_day_2 = data_set[s_id,day2_idx]
            y_day_2 = data_set[s_id,day2_idx+1]
            
            x_day_3 = data_set[s_id,day3_idx]
            y_day_3 = data_set[s_id,day3_idx+1]
            
            matrix_for_regression = np.array([[x_day_1,y_day_1],[x_day_2,y_day_2],[x_day_3,y_day_3]])
            
            
            betas = compute_regression_coeffs(matrix_for_regression)
            
            true_y = matrix_for_regression[:,1:2]
            x = matrix_for_regression[:,0:1]
        
            pred_y = betas[0] + betas[1]*x
            
            cond_var = get_cond_variance(true_y,pred_y)
            
            modelled_matrix[s_id,time_stamp+1] = (betas[0],betas[1],cond_var)
        
    return modelled_matrix
            
    

In [15]:
dm = get_data_matrix('intelTemperatureTrain.csv')
mod_mat = get_model_2_matrix(dm,3)
mod_mat[49][5]

(0.35733590733590681, 0.97490347490347484, 0.0018597168597168426)

In [16]:
dm = \
np.array([[1.62E+01,1.62E+01],
[1.45E+01,1.45E+01],
[1.60E+01,1.59E+01]])

clf = LinearRegression()
clf.fit(dm[:,0:1],dm[:,1:2])


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

##Inference

####Window Inference:
for every column<br>
{<br>
  Recompute $\mu$ based on previous column's $\mu$<br>
  to_be_predicted = {All - ones in window}
  set values for to_be_predicted = $\mu$  <br>
}

In [17]:
def chunks(_list,size):
    '''Get Chunks of size size from a list _list '''
    for x in xrange(0, len(_list),size):
        yield _list[x:x+size]

In [18]:
def get_windows(no_of_sensors,no_of_cols,budget):    
    '''Get the window indices for each column/each time-stamp'''
    pool = cycle(range(0,no_of_sensors))

    final_list = []

    if budget == 0 :
        
        for _ in range(0,no_of_cols):
            final_list.append([])
            
        return final_list
    
    else:
        
        for _id,x in enumerate(pool):
            if _id == budget * no_of_cols:
                break
            final_list.append(x)

        return list(chunks(final_list, budget))

In [19]:
get_windows(4,5,3)

[[0, 1, 2], [3, 0, 1], [2, 3, 0], [1, 2, 3], [0, 1, 2]]

### Build Prediction matrix for Model 1 using Window Sliding

In [44]:
def model_1_build_pred_mat_using_sliding(test_matrix,budget,sensor_params):
    '''Predict the ones that are not in window. For the rest, use test data'''
    total_cols = test_matrix.shape[1]
    total_rows = test_matrix.shape[0]

    
    windows = get_windows(total_rows,total_cols,budget)

    sensor_params = np.array(sensor_params)
    
    prev_col_means = sensor_params[:,0:1]    
    beta_0 = sensor_params[:,2:3]
    beta_1 = sensor_params[:,3:4]
    
    for col in range(0,total_cols):        
        
        if col >0 :                
                prev_col_means = beta_0 + (beta_1*prev_col_means)
                
        pred_row_ids = set(range(0,total_rows)) - set(windows[col])

        for r_id in pred_row_ids:            
            
            test_matrix[r_id,col] = prev_col_means[r_id]
    
    return test_matrix

In [46]:
test_matrix = np.array([[1,2,22],[3,4,23],[5,6,24]])
sensor_params = [(7,8,9,10,11),(12,13,14,15,16),(17,18,19,20,21)]
budget = 2
model_1_build_pred_mat_using_sliding(test_matrix,budget,sensor_params)

array([[  1,   2, 799],
       [  3, 194,  23],
       [ 17,   6,  24]])

In [113]:

for budget in [0, 5, 10, 20, 25]:
    
    #Train
    dm = get_data_matrix('intelTemperatureTrain.csv')
    sensor_params = get_model_1_params(dm)

    #Test
    tm = get_data_matrix('intelTemperatureTest.csv')
    pred_mat = model_1_build_pred_mat_using_sliding(tm,budget,sensor_params)
    pred_mat

    #Evaluate
    tm = get_data_matrix('intelTemperatureTest.csv')
    print mean_abs_error(tm,pred_mat)


In [51]:
def mean_abs_error(actual_mat,pred_mat):
    '''Compute the mean absolute error'''
    diff_mat = np.subtract(actual_mat,pred_mat)
    diff_mat = np.fabs(diff_mat)
    
    total_cells = diff_mat.shape[0] * diff_mat.shape[1]
    
    mean_abs_error = float(np.sum(diff_mat))/ total_cells
    
    return mean_abs_error

In [53]:
tm = get_data_matrix('intelTemperatureTest.csv')
mean_abs_error(tm,pred_mat)

1.4281901189194444

In [125]:

for budget in [0, 5, 10, 20, 25]:
    
    #Train
    dm = get_data_matrix('intelHumidityTrain.csv')
    sensor_params = get_model_1_params(dm)

    #Test
    tm = get_data_matrix('intelHumidityTest.csv')
    pred_mat = model_1_build_pred_mat_using_sliding(tm,budget,sensor_params)
    pred_mat

    #Evaluate
    tm = get_data_matrix('intelHumidityTest.csv')
    print budget, ',', mean_abs_error(tm,pred_mat)


0 , 5.36506753365
5 , 4.82813742104
10 , 4.30431623368
20 , 3.23055707571
25 , 2.6813335661


In [122]:

for budget in [0, 5, 10, 20, 25]:
    
    #Train
    dm = get_data_matrix('intelHumidityTrain.csv')
    sensor_params = get_model_1_params(dm)

    #Test
    tm = get_data_matrix('intelHumidityTest.csv')
    pred_mat = model_1_build_pred_mat_using_sliding(tm,budget,sensor_params)
    pred_mat

    #Evaluate
    tm = get_data_matrix('intelHumidityTest.csv')
    print budget, ',', mean_abs_error(tm,pred_mat)


0 , 5.36506753365
5 , 4.82813742104
10 , 4.30431623368
20 , 3.23055707571
25 , 2.6813335661


### Build Prediction matrix for Model 2 using Window Sliding

In [54]:
dm = get_data_matrix('intelTemperatureTrain.csv')
model_2_matrix = get_model_2_matrix(dm,3)
model_2_matrix.shape

(50L, 48L)

In [84]:
def construct_mean_matrix(modelled_matrix):
    
    rows = modelled_matrix.shape[0]
    cols = modelled_matrix.shape[1]
    
    mean_matrix = np.zeros((rows,cols),dtype=float)
    for row in range(0,rows):
        for col in range(0,cols):
            if col == 0:
                mean_matrix[row,col] = modelled_matrix[row][col][0]
            else:
                beta_0 = modelled_matrix[row][col][0]
                beta_1 = modelled_matrix[row][col][1]
                                
                prev_mean = mean_matrix[row][col-1]
                
                mean_matrix[row,col] = beta_0 + (beta_1 * prev_mean)
    return mean_matrix

In [87]:
def model_2_build_pred_mat_using_sliding(test_matrix,budget,model_2_matrix,max_cols=48):
    '''Predict the ones that are not in window. For the rest, use test data'''
    total_cols = test_matrix.shape[1]
    total_rows = test_matrix.shape[0]

    mean_matrix = construct_mean_matrix(model_2_matrix)
    
    windows = get_windows(total_rows,total_cols,budget)

    for col in range(0,total_cols):
        
        if col >= max_cols:
            mod_col = col % max_cols
        else:
            mod_col = col
        #Row Ids to be predicted
        pred_row_ids = set(range(0,total_rows)) - set(windows[col])
        

        for r_id in pred_row_ids:            
            
            test_matrix[r_id,col] =  mean_matrix[r_id,mod_col]      
    
    return test_matrix

In [88]:
train_matrix = np.array([[1,2,7],[3,4,8],[5,6,9]])
sensor_params = np.array([[(1,2),(7,8,9),(16,17,18)],[(3,4),(10,11,12),(19,20,21)],[(5,6),(13,14,15),(22,23,24)]])
print sensor_params
test_matrix = np.array([[50,51,52],[53,54,55],[56,57,58]])
budget = 2
model_2_build_pred_mat_using_sliding(test_matrix,budget,sensor_params)

[[(1, 2) (7, 8, 9) (16, 17, 18)]
 [(3, 4) (10, 11, 12) (19, 20, 21)]
 [(5, 6) (13, 14, 15) (22, 23, 24)]]


array([[ 50,  51, 271],
       [ 53,  43,  55],
       [  5,  57,  58]])

In [115]:
for budget in [0, 5, 10, 20, 25]:
    
    #Train
    dm = get_data_matrix('intelHumidityTrain.csv')
    sensor_params = get_model_2_matrix(dm,3)

    #Test
    tm = get_data_matrix('intelHumidityTest.csv')
    pred_mat = model_2_build_pred_mat_using_sliding(tm,budget,sensor_params)

    #Generate the output file
    row_ids = [x for x in range(0,50)] #50 sensors

    col_ids = list(np.arange(0.5,24.5,0.5))
    col_ids += col_ids
    col_ids = [x if x!= 24 else 0 for x in col_ids]

    df = pd.DataFrame(pred_mat, index=row_ids, columns=col_ids)
    file_name = 'd-w' + str(budget) + '.csv'
    df.to_csv(file_name, index=True, header=True, sep=',')

'''
#Train
dm = get_data_matrix('intelTemperatureTrain.csv')
sensor_params = get_model_2_matrix(dm,3)

#Test
tm = get_data_matrix('intelTemperatureTest.csv')
pred_mat = model_2_build_pred_mat_using_sliding(tm,25,sensor_params)

#Evaluate
tm = get_data_matrix('intelTemperatureTest.csv')
mean_abs_error(tm,pred_mat)'''

"\n#Train\ndm = get_data_matrix('intelTemperatureTrain.csv')\nsensor_params = get_model_2_matrix(dm,3)\n\n#Test\ntm = get_data_matrix('intelTemperatureTest.csv')\npred_mat = model_2_build_pred_mat_using_sliding(tm,25,sensor_params)\n\n#Evaluate\ntm = get_data_matrix('intelTemperatureTest.csv')\nmean_abs_error(tm,pred_mat)"

In [126]:
for budget in [0, 5, 10, 20, 25]:
    
    #Train
    dm = get_data_matrix('intelHumidityTrain.csv')
    sensor_params = get_model_2_matrix(dm,3)

    #Test
    tm = get_data_matrix('intelHumidityTest.csv')
    pred_mat = model_2_build_pred_mat_using_sliding(tm,budget,sensor_params)
    

    #Evaluate
    tm = get_data_matrix('intelHumidityTest.csv')
    print budget, ',', mean_abs_error(tm,pred_mat)


0 , 3.47049305556
5 , 3.11869444444
10 , 2.78196527778
20 , 2.06980555556
25 , 1.75699305556


In [123]:
for budget in [0, 5, 10, 20, 25]:
    
    #Train
    dm = get_data_matrix('intelHumidityTrain.csv')
    sensor_params = get_model_2_matrix(dm,3)

    #Test
    tm = get_data_matrix('intelHumidityTest.csv')
    pred_mat = model_2_build_pred_mat_using_sliding(tm,budget,sensor_params)
    

    #Evaluate
    tm = get_data_matrix('intelHumidityTest.csv')
    print budget, ',', mean_abs_error(tm,pred_mat)


0 , 3.47049305556
5 , 3.11869444444
10 , 2.78196527778
20 , 2.06980555556
25 , 1.75699305556


In [102]:
#(mu_i,var_i,b_0,b_1,sigma**2)

def model_1_test_matrix(model_1_params, test_matrix):
    
    total_cols = test_matrix.shape[1]
    total_rows = test_matrix.shape[0]
    
    model_2_mat = np.zeros((total_rows,total_cols),dtype=list)
    
    for r_id in range(0,total_rows):
        
        for c_id in range(0,total_cols):
            
            sensor_params = model_1_params[r_id]
            mu_i = sensor_params[0]
            sigma_i = sensor_params[1]
            b_0 = sensor_params[2]
            b_1 = sensor_params[3]
            cond_var = sensor_params[4]
            
            if c_id == 0:                
                model_2_mat[r_id,c_id] = [mu_i,sigma_i]            
            else:
                mu = b_0 + (b_1*model_2_mat[r_id,c_id-1][0])
                sigma = cond_var + ((b_1**2)*(model_2_mat[r_id,c_id-1][1]))
                
                model_2_mat[r_id,c_id] = [mu_i,sigma_i]
    
    return model_2_mat

In [107]:
tm = get_data_matrix('intelTemperatureTest.csv')
modelled_mat = model_1_test_matrix(model_1_params, tm)

model_2_mat = model_1_test_matrix(model_1_params, tm)
model_2_mat.shape

(50L, 96L)

In [108]:
def top_k_indices(_list,k):
    '''Return the indices of top k elements of the list'''
    return sorted(range(len(_list)), key=lambda i: _list[i], reverse=True)[:k]

In [109]:

def get_top_var_indices(mod_mat,budget):
    '''Extract the indices of cells with top k budgets'''
    total_cols = mod_mat.shape[1]
    
    indices_list = []
    for col_id in range(0,total_cols):
        
        #Extract Mu, sigma for a particular column
        cur_col = mod_mat[:,col_id]
        
        #Extract only sigma from cu_col
        sigma_col = [x[1] for x in cur_col]
        
        top_indices = top_k_indices(sigma_col,budget)
        
        indices_list.append(top_indices)
    
    return indices_list

In [110]:
def build_pred_mat_using_Variance(test_matrix,budget,modelled_matrix,max_cols=48):
    '''Predict the ones that are not among the top budget variances. For the rest, use existing data'''
    total_cols = test_matrix.shape[1]
    total_rows = test_matrix.shape[0]

    top_var = get_top_var_indices(modelled_matrix,budget)

    
    for col in range(0,total_cols):
        
        #Row Ids to be predicted
        pred_row_ids = set(range(0,total_rows)) - set(top_var[col])
        
            
        for r_id in pred_row_ids:
            #Assign Mu
            test_matrix[r_id,col] = modelled_matrix[r_id][col][0]
    
    return test_matrix